In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from scipy.spatial.distance import cosine
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import euclidean, cityblock
import nltk
import seaborn as sns
from tqdm import tqdm

# Toggle on-off code text  
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()">
<input type="submit" value="Click to toggle on/off the raw code.">
</form>''')

### Function Definitions

In [27]:
from scipy.spatial.distance import euclidean

def nearest_k(query, objects, k, dist):
    """Return the indices to objects most similar to query
    
    Parameters
    ----------
    query : ndarray
        query object represented in the same form vector representation as the
        objects
    objects : ndarray
        vector-represented objects in the database; rows correspond to 
        objects, columns correspond to features
    k : int
        number of most similar objects to return
    dist : function
        accepts two ndarrays as parameters then returns their distance
    
    Returns
    -------
    ndarray
        Indices to the most similar objects in the database
    """
    return np.argsort([dist(query, o) for o in objects])[:k]

def inverse_transform(text_query):
    """Codes copied from data_building_and_cleaning. Credits to the 
    rightful owner."""
    
    import string
    punc = string.punctuation.replace('-', '').replace("'", '')
    table = str.maketrans(' ', ' ', punc)
    stripped = [w.translate(table) for w in [text_query.lower()]]

    ######################################################################

    # filter out stop words
    from nltk.corpus import stopwords

    stop_words = set(stopwords.words('english'))

    no_stop_words = []
    for word_list in stripped:
        word_list = word_list.split()
        words = [w for w in word_list if not w in stop_words]
        no_stop_words += [words]

    ######################################################################

    # nltk.download('wordnet')
    from nltk.stem import WordNetLemmatizer 

    lemmatizer = WordNetLemmatizer() 

    lematized = []
    for word_list in no_stop_words:
        words = [lemmatizer.lemmatize(w) for w in word_list]
        lematized += [words]

    title_cleaned = []
    for i in lematized:
        title_cleaned+=[" ".join(i)]

    return title_cleaned[0]

def IR_search(text_query, k, congress, criteria):
    from collections import defaultdict
    
    pres = {13:defaultdict(list), 14:defaultdict(list), 15:defaultdict(list), 
        16:defaultdict(list), 17:defaultdict(list), 18:defaultdict(list)}
    rec = {13:defaultdict(list), 14:defaultdict(list), 15:defaultdict(list), 
        16:defaultdict(list), 17:defaultdict(list), 18:defaultdict(list)}

    # congess filed_by scope filed_on primary_committee
    filtered = df_clean.query(f"congress == {congress} ").copy()

    tfidf = TfidfVectorizer(stop_words='english', min_df=0.001, max_df=0.999)
    tfidf.fit(filtered['combined_title'])
    X_idf  = tfidf.transform(filtered['combined_title']).toarray()
    X_feat_names = tfidf.get_feature_names()

    query = [inverse_transform(text_query)]                    
    print(f"Search term: {query[0]}")                          
    search_results = nearest_k(tfidf.transform(query).toarray()[0], X_idf, k, 
                               euclidean)
    pd.set_option('display.max_colwidth', None)
    print(f'Top {k} search results:\n\n')
    
    congress_ = list(range(13,19))
    
    for s in range(50):  # 50 is arbitrary, can be changed
        nearest = filtered.iloc[nearest_k(tfidf.transform(query).toarray()[0], 
                                          X_idf, 5, euclidean)]
        r = np.cumsum(nearest[criteria] == filtered[criteria].iloc[s])
        precision = r/np.arange(1, len(r)+1)
        n = sum(filtered[criteria] == filtered[criteria].iloc[s])
        recall = r/n
        recall = [0] + recall.tolist()
        precision = [1] + precision.tolist()
        pres[congress][s].append(precision)
        rec[congress][s].append(recall)

    ################################################################

    print(f'{text_query} precision_({criteria}): %.3f' 
          %np.mean([np.mean(pres[congress][key][0]) for key 
                    in pres[congress].keys()]))
    print(f'{text_query} recall_({criteria}): %.5f'
          %(np.mean([np.mean(rec[congress][key][0]) for key in 
                     rec[congress].keys()])))

    return filtered.iloc[search_results]

df_clean = pd.read_pickle('../data_cleaning/clusters_added.pkl')

bill_num = None
filed_by = None
scope = None
min_file_date = None
max_file_date = None
primary_committee = None

### 1st query: `minimum wage`

#### relevant criteria: `scope`

In [28]:
## sample querying

text_query = 'Minimum Wage'   ## type a sample query
k = 5                         ## specify no. of results
congress = 14                 ## specify which congress
criteria = 'scope'
IR_search(text_query, k, congress, criteria)

Search term: minimum wage
Top 5 search results:


Minimum Wage precision_(scope): 1.000
Minimum Wage recall_(scope): 0.00102


,congress,type,bill_num,title,filed_on,filed_by,long_title,scope,subjects,primary_committee,combined_title,clustering_ward_a,clustering_ward_b,clustering_ward_c
2783,14,Senate,103,INDIVIDUAL TAX EXEMPTION (MINIMUM WAGE EARNER),2007-06-30,"[Roxas, Mar]","AN ACT EXEMPTING FROM INDIVIDUAL TAX WORKERS IN THE PRIVATE SECTOR RECEIVING MINIMUM WAGE, AS WELL AS GOVERNMENT WORKERS IN SALARY GRADES 1 TO 3, AMENDING FOR THIS PURPOSE REPUBLIC ACT 8424, AND FOR OTHER PURPOSES",National,Individual Income Tax ExemptionMinimum Wage Earners (Individual Income Tax Exemption),Ways and Means,individual tax exemption minimum wage earner act exempting individual tax worker private sector receiving minimum wage well government worker salary grade 1 3 amending purpose republic act 8424 purpose,6,14,59
4256,14,Senate,1576,MINIMUM WAGE EARNERS INCOME TAX EXEMPTION,2007-09-12,"[Villar, Manny]","AN ACT EXEMPTING MINIMUM WAGE EARNERS FROM PAYMENT OF INCOME TAX AMENDING FOR THE PURPOSE RELEVANT PROVISIONS OF THE NATIONAL INTERNAL REVENUE CODE OF 1997, AS AMENDED, AND FOR OTHER PURPOSES",National,Individual Income Tax ExemptionNational Internal Revenue Code (NIRC),Ways and Means,minimum wage earner income tax exemption act exempting minimum wage earner payment income tax amending purpose relevant provision national internal revenue code 1997 amended purpose,6,14,59
4857,14,Senate,2177,INCOME TAX EXEMPTION FOR MINIMUM WAGE EARNERS,2008-04-21,"[Zubiri, Juan Miguel F, ]","AN ACT EXEMPTING MINIMUM WAGE EARNERS FROM PAYMENT OF INCOME TAX AMENDING FOR THE PURPOSE RELEVANT PROVISIONS OF THE NATIONAL INTERNAL REVENUE CODE OF 1997, AS AMENDED, AND FOR OTHER PURPOSES",National,Individual Income Tax (Taxpayers Income Tax Rates),Ways and Means,income tax exemption minimum wage earner act exempting minimum wage earner payment income tax amending purpose relevant provision national internal revenue code 1997 amended purpose,6,14,59
3259,14,Senate,579,REGIONAL BOARD TO ISSUE WAGE ORDER ANNUALLY ON OR BEFORE MAY 1,2007-07-02,"[Ejercito-Estrada, Jinggoy P, ]","AN ACT AMENDING ARTICLE 123 OF PRESIDENTIAL DECREE NO. 442, AS AMENDED, OTHERWISE KNOWN AS THE LABOR CODE OF THE PHILIPPINES, BY REQUIRING THE REGIONAL BOARD TO ISSUE A WAGE ORDER EVERY YEAR OR AN EXPLANATION FOR NOT ISSUING SUCH WAGE ORDER",National,Wage Order (Regional Board)Labor Code (Pd 442),"Labor, Employment and Human Resources Development",regional board issue wage order annually may 1 act amending article 123 presidential decree 442 amended otherwise known labor code philippine requiring regional board issue wage order every year explanation issuing wage order,0,2,1
4073,14,Senate,1393,LIVING WAGE ACT OF 2007,2007-07-26,"[Legarda, Loren B, ]","AN ACT DEFINING A LIVING WAGE FOR EMPLOYEES IN THE PUBLIC AND PRIVATE SECTORS, ESTABLISHING THE MECHANISM AND PROPER STANDARDS THEREFOR, APPROPRIATING FUNDS THEREFOR AND FOR OTHER PURPOSES",National,Living WagePrivate Sector,"Labor, Employment and Human Resources Development",living wage act 2007 act defining living wage employee public private sector establishing mechanism proper standard therefor appropriating fund therefor purpose,0,2,1


#### relevant criteria: `primary committee`

In [29]:
## sample querying

text_query = 'Minimum Wage'   ## type a sample query
k = 5                         ## specify no. of results
congress = 14                 ## specify which congress
criteria = 'primary_committee'
IR_search(text_query, k, congress, criteria)

Search term: minimum wage
Top 5 search results:


Minimum Wage precision_(primary_committee): 0.214
Minimum Wage recall_(primary_committee): 0.00202


,congress,type,bill_num,title,filed_on,filed_by,long_title,scope,subjects,primary_committee,combined_title,clustering_ward_a,clustering_ward_b,clustering_ward_c
2783,14,Senate,103,INDIVIDUAL TAX EXEMPTION (MINIMUM WAGE EARNER),2007-06-30,"[Roxas, Mar]","AN ACT EXEMPTING FROM INDIVIDUAL TAX WORKERS IN THE PRIVATE SECTOR RECEIVING MINIMUM WAGE, AS WELL AS GOVERNMENT WORKERS IN SALARY GRADES 1 TO 3, AMENDING FOR THIS PURPOSE REPUBLIC ACT 8424, AND FOR OTHER PURPOSES",National,Individual Income Tax ExemptionMinimum Wage Earners (Individual Income Tax Exemption),Ways and Means,individual tax exemption minimum wage earner act exempting individual tax worker private sector receiving minimum wage well government worker salary grade 1 3 amending purpose republic act 8424 purpose,6,14,59
4256,14,Senate,1576,MINIMUM WAGE EARNERS INCOME TAX EXEMPTION,2007-09-12,"[Villar, Manny]","AN ACT EXEMPTING MINIMUM WAGE EARNERS FROM PAYMENT OF INCOME TAX AMENDING FOR THE PURPOSE RELEVANT PROVISIONS OF THE NATIONAL INTERNAL REVENUE CODE OF 1997, AS AMENDED, AND FOR OTHER PURPOSES",National,Individual Income Tax ExemptionNational Internal Revenue Code (NIRC),Ways and Means,minimum wage earner income tax exemption act exempting minimum wage earner payment income tax amending purpose relevant provision national internal revenue code 1997 amended purpose,6,14,59
4857,14,Senate,2177,INCOME TAX EXEMPTION FOR MINIMUM WAGE EARNERS,2008-04-21,"[Zubiri, Juan Miguel F, ]","AN ACT EXEMPTING MINIMUM WAGE EARNERS FROM PAYMENT OF INCOME TAX AMENDING FOR THE PURPOSE RELEVANT PROVISIONS OF THE NATIONAL INTERNAL REVENUE CODE OF 1997, AS AMENDED, AND FOR OTHER PURPOSES",National,Individual Income Tax (Taxpayers Income Tax Rates),Ways and Means,income tax exemption minimum wage earner act exempting minimum wage earner payment income tax amending purpose relevant provision national internal revenue code 1997 amended purpose,6,14,59
3259,14,Senate,579,REGIONAL BOARD TO ISSUE WAGE ORDER ANNUALLY ON OR BEFORE MAY 1,2007-07-02,"[Ejercito-Estrada, Jinggoy P, ]","AN ACT AMENDING ARTICLE 123 OF PRESIDENTIAL DECREE NO. 442, AS AMENDED, OTHERWISE KNOWN AS THE LABOR CODE OF THE PHILIPPINES, BY REQUIRING THE REGIONAL BOARD TO ISSUE A WAGE ORDER EVERY YEAR OR AN EXPLANATION FOR NOT ISSUING SUCH WAGE ORDER",National,Wage Order (Regional Board)Labor Code (Pd 442),"Labor, Employment and Human Resources Development",regional board issue wage order annually may 1 act amending article 123 presidential decree 442 amended otherwise known labor code philippine requiring regional board issue wage order every year explanation issuing wage order,0,2,1
4073,14,Senate,1393,LIVING WAGE ACT OF 2007,2007-07-26,"[Legarda, Loren B, ]","AN ACT DEFINING A LIVING WAGE FOR EMPLOYEES IN THE PUBLIC AND PRIVATE SECTORS, ESTABLISHING THE MECHANISM AND PROPER STANDARDS THEREFOR, APPROPRIATING FUNDS THEREFOR AND FOR OTHER PURPOSES",National,Living WagePrivate Sector,"Labor, Employment and Human Resources Development",living wage act 2007 act defining living wage employee public private sector establishing mechanism proper standard therefor appropriating fund therefor purpose,0,2,1


### 2nd query: `Overseas Filipino Workers`

#### relevant criteria: `scope`

In [30]:
## sample querying

text_query = 'Overseas Filipino Workers'   ## type a sample query
k = 5                         ## specify no. of results
congress = 14                 ## specify which congress
criteria = 'scope'
IR_search(text_query, k, congress, criteria)

Search term: overseas filipino worker
Top 5 search results:


Overseas Filipino Workers precision_(scope): 1.000
Overseas Filipino Workers recall_(scope): 0.00102


,congress,type,bill_num,title,filed_on,filed_by,long_title,scope,subjects,primary_committee,combined_title,clustering_ward_a,clustering_ward_b,clustering_ward_c
5008,14,Senate,2328,OVERSEAS FILIPINO WORKERS PENSION FUND ACT OF 2008,2008-05-27,"[Trillanes IV, Antonio ""Sonny"" F, ]",AN ACT CREATING THE OVERSEAS FILIPINO WORKERS PENSION AND APPROPRIATING FUNDS THEREFOR,National,Overseas Contract Workers (OCWS)/Overseas Filipino Workers (OFWs),"Labor, Employment and Human Resources Development",overseas filipino worker pension fund act 2008 act creating overseas filipino worker pension appropriating fund therefor,0,17,72
2834,14,Senate,154,REVISED MIGRANT WORKERS AND OVERSEAS FILIPINOS ACT OF 2007,2007-06-30,"[Ejercito-Estrada, Jinggoy P, ]","AN ACT AMENDING REPUBLIC ACT NO. 8042, OTHERWISE KNOWN AS THE MIGRANT WORKERS AND OVERSEAS FILIPINOS ACT OF 1995, FURTHER IMPROVING THE HIGHER STANDARDS OF PROTECTION AND PROMOTION OF THE WELFARE OF MIGRANT WORKERS, THEIR FAMILIES AND OVERSEAS FILIPINOS IN DISTRESS, AND FOR OTHER PURPOSES",National,Revised Migrant Workers and Overseas Filipinos Act of 2007Overseas Contract Workers (OCWS)/Overseas Filipino Workers (OFWs),"Labor, Employment and Human Resources Development",revised migrant worker overseas filipino act 2007 act amending republic act 8042 otherwise known migrant worker overseas filipino act 1995 improving higher standard protection promotion welfare migrant worker family overseas filipino distress purpose,0,17,72
2734,14,Senate,54,"MIGRANT WORKERS AND OVERSEAS FILIPINOS, AMENDMENTS",2007-06-30,"[Lapid, Manuel ""Lito"" M, ]","AN ACT PROVIDING RETIREMENT, INSURANCE, HEALTH AND EDUCATIONAL BENEFITS TO OVERSEAS CONTRACT WORKERS (OCWS) AND THEIR BENEFICIARIES AMENDING FOR THE PURPOSE CERTAIN PROVISIONS OF REPUBLIC ACT NO. 8042 OTHERWISE KNOWN AS ‘MIGRANT WORKERS AND OVERSEAS FILIPINOS ACT OF 1995’, AND FOR OTHER PURPOSES",National,"Overseas Contract Workers (OCWS)/Overseas Filipino Workers (OFWs)Migrant Workers and Overseas Filipinos (RA 8042, Amendments)","Labor, Employment and Human Resources Development",migrant worker overseas filipino amendment act providing retirement insurance health educational benefit overseas contract worker ocws beneficiary amending purpose certain provision republic act 8042 otherwise known ‘migrant worker overseas filipino act 1995’ purpose,0,17,72
5378,14,Senate,2698,MANDATORY EMPLOYERS' INSURANCE FOR OVERSEAS FILIPINO WORKERS ACT OF 2008,2008-11-06,"[Villar, Manny]","AN ACT TO PROVIDE A MANDATORY EMPLOYERS' INSURANCE FOR ALL LEGALLY EMPLOYED OVERSEAS FILIPINO WORKERS, PROVIDING MECHANISMS THEREFOR AND FOR OTHER PURPOSES",National,Overseas Contract Workers (OCWS)/Overseas Filipino Workers (OFWs),"Labor, Employment and Human Resources Development",mandatory employers' insurance overseas filipino worker act 2008 act provide mandatory employers' insurance legally employed overseas filipino worker providing mechanism therefor purpose,0,17,72
5597,14,Senate,2917,OVERSEAS FILIPINO WORKERS,2008-11-27,"[Villar, Manny]","AN ACT GRANTING BENEFITS AND SPECIAL PRIVILEGES TO FORMER OVERSEAS FILIPINO WORKERS, AND FOR OTHER PURPOSES",National,Overseas Contract Workers (OCWS)/Overseas Filipino Workers (OFWs),"Labor, Employment and Human Resources Development",act granting benefit special privilege former overseas filipino worker purpose,0,1,17


#### relevant criteria: `primary_committee`

In [31]:
## sample querying

text_query = 'Overseas Filipino Workers'   ## type a sample query
k = 5                         ## specify no. of results
congress = 14                 ## specify which congress
criteria = 'primary_committee'
IR_search(text_query, k, congress, criteria)

Search term: overseas filipino worker
Top 5 search results:


Overseas Filipino Workers precision_(primary_committee): 0.200
Overseas Filipino Workers recall_(primary_committee): 0.00071


,congress,type,bill_num,title,filed_on,filed_by,long_title,scope,subjects,primary_committee,combined_title,clustering_ward_a,clustering_ward_b,clustering_ward_c
5008,14,Senate,2328,OVERSEAS FILIPINO WORKERS PENSION FUND ACT OF 2008,2008-05-27,"[Trillanes IV, Antonio ""Sonny"" F, ]",AN ACT CREATING THE OVERSEAS FILIPINO WORKERS PENSION AND APPROPRIATING FUNDS THEREFOR,National,Overseas Contract Workers (OCWS)/Overseas Filipino Workers (OFWs),"Labor, Employment and Human Resources Development",overseas filipino worker pension fund act 2008 act creating overseas filipino worker pension appropriating fund therefor,0,17,72
2834,14,Senate,154,REVISED MIGRANT WORKERS AND OVERSEAS FILIPINOS ACT OF 2007,2007-06-30,"[Ejercito-Estrada, Jinggoy P, ]","AN ACT AMENDING REPUBLIC ACT NO. 8042, OTHERWISE KNOWN AS THE MIGRANT WORKERS AND OVERSEAS FILIPINOS ACT OF 1995, FURTHER IMPROVING THE HIGHER STANDARDS OF PROTECTION AND PROMOTION OF THE WELFARE OF MIGRANT WORKERS, THEIR FAMILIES AND OVERSEAS FILIPINOS IN DISTRESS, AND FOR OTHER PURPOSES",National,Revised Migrant Workers and Overseas Filipinos Act of 2007Overseas Contract Workers (OCWS)/Overseas Filipino Workers (OFWs),"Labor, Employment and Human Resources Development",revised migrant worker overseas filipino act 2007 act amending republic act 8042 otherwise known migrant worker overseas filipino act 1995 improving higher standard protection promotion welfare migrant worker family overseas filipino distress purpose,0,17,72
2734,14,Senate,54,"MIGRANT WORKERS AND OVERSEAS FILIPINOS, AMENDMENTS",2007-06-30,"[Lapid, Manuel ""Lito"" M, ]","AN ACT PROVIDING RETIREMENT, INSURANCE, HEALTH AND EDUCATIONAL BENEFITS TO OVERSEAS CONTRACT WORKERS (OCWS) AND THEIR BENEFICIARIES AMENDING FOR THE PURPOSE CERTAIN PROVISIONS OF REPUBLIC ACT NO. 8042 OTHERWISE KNOWN AS ‘MIGRANT WORKERS AND OVERSEAS FILIPINOS ACT OF 1995’, AND FOR OTHER PURPOSES",National,"Overseas Contract Workers (OCWS)/Overseas Filipino Workers (OFWs)Migrant Workers and Overseas Filipinos (RA 8042, Amendments)","Labor, Employment and Human Resources Development",migrant worker overseas filipino amendment act providing retirement insurance health educational benefit overseas contract worker ocws beneficiary amending purpose certain provision republic act 8042 otherwise known ‘migrant worker overseas filipino act 1995’ purpose,0,17,72
5378,14,Senate,2698,MANDATORY EMPLOYERS' INSURANCE FOR OVERSEAS FILIPINO WORKERS ACT OF 2008,2008-11-06,"[Villar, Manny]","AN ACT TO PROVIDE A MANDATORY EMPLOYERS' INSURANCE FOR ALL LEGALLY EMPLOYED OVERSEAS FILIPINO WORKERS, PROVIDING MECHANISMS THEREFOR AND FOR OTHER PURPOSES",National,Overseas Contract Workers (OCWS)/Overseas Filipino Workers (OFWs),"Labor, Employment and Human Resources Development",mandatory employers' insurance overseas filipino worker act 2008 act provide mandatory employers' insurance legally employed overseas filipino worker providing mechanism therefor purpose,0,17,72
5597,14,Senate,2917,OVERSEAS FILIPINO WORKERS,2008-11-27,"[Villar, Manny]","AN ACT GRANTING BENEFITS AND SPECIAL PRIVILEGES TO FORMER OVERSEAS FILIPINO WORKERS, AND FOR OTHER PURPOSES",National,Overseas Contract Workers (OCWS)/Overseas Filipino Workers (OFWs),"Labor, Employment and Human Resources Development",act granting benefit special privilege former overseas filipino worker purpose,0,1,17


### 3rd query: `Illegal Drugs`

#### relevant criteria: `scope`

In [32]:
## sample querying

text_query = 'Illegal Drugs'   ## type a sample query
k = 5                         ## specify no. of results
congress = 14                 ## specify which congress
criteria = 'scope'
IR_search(text_query, k, congress, criteria)

Search term: illegal drug
Top 5 search results:


Illegal Drugs precision_(scope): 1.000
Illegal Drugs recall_(scope): 0.00102


,congress,type,bill_num,title,filed_on,filed_by,long_title,scope,subjects,primary_committee,combined_title,clustering_ward_a,clustering_ward_b,clustering_ward_c
3964,14,Senate,1284,DRUGS DEMAND REDUCTION,2007-07-18,"[Defensor Santiago, Miriam]",AN ACT PROVIDING FOR PROGRAMS TO FACILITATE A SIGNIFICANT REDUCTION IN THE INCIDENCE AND PREVALENCE OF SUBSTANCE ABUSE THROUGH REDUCING THE DEMAND FOR ILLEGAL DRUGS AND THE INAPPROPRIATE USE OF ILLEGAL DRUGS,National,Drug Demand ReductionIllegal Drugs,Public Order and Dangerous Drugs,drug demand reduction act providing program facilitate significant reduction incidence prevalence substance abuse reducing demand illegal drug inappropriate use illegal drug,0,2,85
3996,14,Senate,1316,DRUG DEALER LIABILITY ACT,2007-07-24,"[Defensor Santiago, Miriam]",AN ACT PROVIDING FOR THE CIVIL LIABILITY OF DRUG DEALERS IN FAVOR OF PERSONS INJURED BY ILLEGAL DRUGS,National,Drug Dealer LiabilityIllegal Drugs,Public Order and Dangerous Drugs,drug dealer liability act act providing civil liability drug dealer favor person injured illegal drug,0,2,85
4430,14,Senate,1750,DRUG-FREE BORDERS ACT OF 2007,2007-10-16,"[Defensor Santiago, Miriam]",AN ACT SECURING THE PHILIPPINES' BORDERS TO MAKE MORE EFFECTIVE THE PROHIBITION AGAINST THE ENTRY OF ILLEGAL DRUGS INTO THE COUNTRY,National,Drug-Free Borders,Public Order and Dangerous Drugs,drug-free border act 2007 act securing philippines' border make effective prohibition entry illegal drug country,0,2,85
2733,14,Senate,53,ILLEGAL NUMBERS GAMES,2007-06-30,"[Lapid, Manuel ""Lito"" M, ]","AN ACT AMENDING CERTAIN PROVISIONS OF REPUBLIC ACT NO. 9287 OTHERWISE KNOWN AS ‘AN ACT INCREASING THE PENALTIES OF ILLEGAL NUMBERS GAMES’, BY INCLUDING ALTOGETHER ALL FORMS OF ILLEGAL GAMBLING, CREATING THE ANTI-ILLEGAL GAMBLING BOARD, APPROPRIATING FUNDS THEREFOR AND FOR OTHER PURPOSES",National,Illegal Number Games (RA 9287)Anti-Illegal Gambling Board (Creation),Games and Amusement,act amending certain provision republic act 9287 otherwise known ‘an act increasing penalty illegal number games’ including altogether form illegal gambling creating anti-illegal gambling board appropriating fund therefor purpose,0,2,4
3998,14,Senate,1318,THE DRUG ABUSE PREVENTION PROGRAM IN EDUCATIONAL INSTITUTIONS ACT,2007-07-24,"[Defensor Santiago, Miriam]",AN ACT MANDATING THE ESTABLISHMENT OF A PROGRAM OF EDUCATION DESIGNED TO HELP ALL MEMBERS OF THE COMMUNITY OF AN EDUCATIONAL INSTITUTION AVOID INVOLVEMENT WITH ILLEGAL DRUGS,National,Drug Abuse Prevention Program in Educational Institutions ActIllegal Drugs,"Basic Education, Arts and Culture",drug abuse prevention program educational institution act act mandating establishment program education designed help member community educational institution avoid involvement illegal drug,0,2,4


#### relevant criteria: `primary_committee`

In [33]:
## sample querying

text_query = 'Illegal Drugs'   ## type a sample query
k = 5                         ## specify no. of results
congress = 14                 ## specify which congress
criteria = 'primary_committee'
IR_search(text_query, k, congress, criteria)

Search term: illegal drug
Top 5 search results:


Illegal Drugs precision_(primary_committee): 0.240
Illegal Drugs recall_(primary_committee): 0.00231


,congress,type,bill_num,title,filed_on,filed_by,long_title,scope,subjects,primary_committee,combined_title,clustering_ward_a,clustering_ward_b,clustering_ward_c
3964,14,Senate,1284,DRUGS DEMAND REDUCTION,2007-07-18,"[Defensor Santiago, Miriam]",AN ACT PROVIDING FOR PROGRAMS TO FACILITATE A SIGNIFICANT REDUCTION IN THE INCIDENCE AND PREVALENCE OF SUBSTANCE ABUSE THROUGH REDUCING THE DEMAND FOR ILLEGAL DRUGS AND THE INAPPROPRIATE USE OF ILLEGAL DRUGS,National,Drug Demand ReductionIllegal Drugs,Public Order and Dangerous Drugs,drug demand reduction act providing program facilitate significant reduction incidence prevalence substance abuse reducing demand illegal drug inappropriate use illegal drug,0,2,85
3996,14,Senate,1316,DRUG DEALER LIABILITY ACT,2007-07-24,"[Defensor Santiago, Miriam]",AN ACT PROVIDING FOR THE CIVIL LIABILITY OF DRUG DEALERS IN FAVOR OF PERSONS INJURED BY ILLEGAL DRUGS,National,Drug Dealer LiabilityIllegal Drugs,Public Order and Dangerous Drugs,drug dealer liability act act providing civil liability drug dealer favor person injured illegal drug,0,2,85
4430,14,Senate,1750,DRUG-FREE BORDERS ACT OF 2007,2007-10-16,"[Defensor Santiago, Miriam]",AN ACT SECURING THE PHILIPPINES' BORDERS TO MAKE MORE EFFECTIVE THE PROHIBITION AGAINST THE ENTRY OF ILLEGAL DRUGS INTO THE COUNTRY,National,Drug-Free Borders,Public Order and Dangerous Drugs,drug-free border act 2007 act securing philippines' border make effective prohibition entry illegal drug country,0,2,85
2733,14,Senate,53,ILLEGAL NUMBERS GAMES,2007-06-30,"[Lapid, Manuel ""Lito"" M, ]","AN ACT AMENDING CERTAIN PROVISIONS OF REPUBLIC ACT NO. 9287 OTHERWISE KNOWN AS ‘AN ACT INCREASING THE PENALTIES OF ILLEGAL NUMBERS GAMES’, BY INCLUDING ALTOGETHER ALL FORMS OF ILLEGAL GAMBLING, CREATING THE ANTI-ILLEGAL GAMBLING BOARD, APPROPRIATING FUNDS THEREFOR AND FOR OTHER PURPOSES",National,Illegal Number Games (RA 9287)Anti-Illegal Gambling Board (Creation),Games and Amusement,act amending certain provision republic act 9287 otherwise known ‘an act increasing penalty illegal number games’ including altogether form illegal gambling creating anti-illegal gambling board appropriating fund therefor purpose,0,2,4
3998,14,Senate,1318,THE DRUG ABUSE PREVENTION PROGRAM IN EDUCATIONAL INSTITUTIONS ACT,2007-07-24,"[Defensor Santiago, Miriam]",AN ACT MANDATING THE ESTABLISHMENT OF A PROGRAM OF EDUCATION DESIGNED TO HELP ALL MEMBERS OF THE COMMUNITY OF AN EDUCATIONAL INSTITUTION AVOID INVOLVEMENT WITH ILLEGAL DRUGS,National,Drug Abuse Prevention Program in Educational Institutions ActIllegal Drugs,"Basic Education, Arts and Culture",drug abuse prevention program educational institution act act mandating establishment program education designed help member community educational institution avoid involvement illegal drug,0,2,4


### 4th query: `Death Penalty`

#### relevant criteria: `scope`

In [34]:
## sample querying

text_query = 'Death Penalty'   ## type a sample query
k = 5                         ## specify no. of results
congress = 14                 ## specify which congress
criteria = 'scope'
IR_search(text_query, k, congress, criteria)

Search term: death penalty
Top 5 search results:


Death Penalty precision_(scope): 1.000
Death Penalty recall_(scope): 0.00102


,congress,type,bill_num,title,filed_on,filed_by,long_title,scope,subjects,primary_committee,combined_title,clustering_ward_a,clustering_ward_b,clustering_ward_c
5002,14,Senate,2322,REIMPOSING DEATH PENALTY ON CERTAIN HEINOUS CRIMES,2008-05-26,"[Zubiri, Juan Miguel F, ]","AN ACT REIMPOSING THE PENALTY OF DEATH ON CERTAIN HEINOUS CRIMES, AMENDING FOR THE PURPOSE CERTAIN SECTIONS OF THE REVISED PENAL CODE, AS AMENDED, AND FOR OTHER PURPOSES",National,Revised Penal Code (RA 3815; Amendments)Death PenaltyHeinous Crimes,Justice and Human Rights,reimposing death penalty certain heinous crime act reimposing penalty death certain heinous crime amending purpose certain section revised penal code amended purpose,1,6,42
5665,14,Senate,2985,TOY-INFLICTED INJURIES AND DEATHS,2009-01-15,"[Defensor Santiago, Miriam]",AN ACT TO PREVENT TOY-INFLICTED INJURIES AND DEATHS,National,Toys,"Trade, Commerce and Entrepreneurship",act prevent toy-inflicted injury death,0,2,4
3105,14,Senate,425,DEATH BENEFITS FOR SEAFARERS ACT OF 2007,2007-07-02,"[Ejercito-Estrada, Jinggoy P, ]",AN ACT FIXING THE MINIMUM STANDARD FOR DEATH BENEFITS TO FILIPINO SEAFARERS EMPLOYED OVERSEAS AND ESTABLISHING A MECHANISM AT THE PHILIPPINE OVERSEAS EMPLOYMENT ADMINISTRATION FOR ITS REGULAR REVIEW AND REALIGNMENT TO PREVAILING INTERNATIONAL STANDARDS AND FOR OTHER PURPOSES,National,SeafarersPhilippine Overseas Employment Administration (POEA),"Labor, Employment and Human Resources Development",death benefit seafarer act 2007 act fixing minimum standard death benefit filipino seafarer employed overseas establishing mechanism philippine overseas employment administration regular review realignment prevailing international standard purpose,0,17,72
4873,14,Senate,2193,ANTI-SALVAGING LAW,2008-04-23,"[Trillanes IV, Antonio ""Sonny"" F, ]","AN ACT QUALIFYING SALVAGING OR EXTRAJUDICIAL KILLING BY ANY PUBLIC OFFICER, PERSON IN AUTHORITY OR AGENT OF A PERSON IN AUTHORITY AS A HEINOUS CRIME, IMPOSING THE DEATH PENALTY THEREFOR AND FOR OTHER PURPOSES",National,Heinous Crimes,Justice and Human Rights,anti-salvaging law act qualifying salvaging extrajudicial killing public officer person authority agent person authority heinous crime imposing death penalty therefor purpose,1,6,84
2966,14,Senate,286,SUSPENDING PAYMENT OF ESTATE TAX ON FAMILY HOME BY SURVIVING SPOUSE IN CASE OF DEATH OF SPOUSE,2007-06-30,"[Gordon, Richard ""Dick"" J, ]","AN ACT AMENDING SECTION 86(A) (4) OF REPUBLIC ACT NO. 8424, AS AMENDED, OTHERWISE KNOWN AS THE NATIONAL INTERNAL REVENUE CODE OF 1997",National,"National Internal Revenue Code (NIRC)Family HomeR.A.No. 8424 (Tax Reform Act of 1997), Amendments",Ways and Means,suspending payment estate tax family home surviving spouse case death spouse act amending section 86a 4 republic act 8424 amended otherwise known national internal revenue code 1997,6,14,59


#### relevant criteria: `primary_committee`

In [35]:
## sample querying

text_query = 'Death Penalty'   ## type a sample query
k = 5                         ## specify no. of results
congress = 14                 ## specify which congress
criteria = 'primary_committee'
IR_search(text_query, k, congress, criteria)

Search term: death penalty
Top 5 search results:


Death Penalty precision_(primary_committee): 0.201
Death Penalty recall_(primary_committee): 0.00048


,congress,type,bill_num,title,filed_on,filed_by,long_title,scope,subjects,primary_committee,combined_title,clustering_ward_a,clustering_ward_b,clustering_ward_c
5002,14,Senate,2322,REIMPOSING DEATH PENALTY ON CERTAIN HEINOUS CRIMES,2008-05-26,"[Zubiri, Juan Miguel F, ]","AN ACT REIMPOSING THE PENALTY OF DEATH ON CERTAIN HEINOUS CRIMES, AMENDING FOR THE PURPOSE CERTAIN SECTIONS OF THE REVISED PENAL CODE, AS AMENDED, AND FOR OTHER PURPOSES",National,Revised Penal Code (RA 3815; Amendments)Death PenaltyHeinous Crimes,Justice and Human Rights,reimposing death penalty certain heinous crime act reimposing penalty death certain heinous crime amending purpose certain section revised penal code amended purpose,1,6,42
5665,14,Senate,2985,TOY-INFLICTED INJURIES AND DEATHS,2009-01-15,"[Defensor Santiago, Miriam]",AN ACT TO PREVENT TOY-INFLICTED INJURIES AND DEATHS,National,Toys,"Trade, Commerce and Entrepreneurship",act prevent toy-inflicted injury death,0,2,4
3105,14,Senate,425,DEATH BENEFITS FOR SEAFARERS ACT OF 2007,2007-07-02,"[Ejercito-Estrada, Jinggoy P, ]",AN ACT FIXING THE MINIMUM STANDARD FOR DEATH BENEFITS TO FILIPINO SEAFARERS EMPLOYED OVERSEAS AND ESTABLISHING A MECHANISM AT THE PHILIPPINE OVERSEAS EMPLOYMENT ADMINISTRATION FOR ITS REGULAR REVIEW AND REALIGNMENT TO PREVAILING INTERNATIONAL STANDARDS AND FOR OTHER PURPOSES,National,SeafarersPhilippine Overseas Employment Administration (POEA),"Labor, Employment and Human Resources Development",death benefit seafarer act 2007 act fixing minimum standard death benefit filipino seafarer employed overseas establishing mechanism philippine overseas employment administration regular review realignment prevailing international standard purpose,0,17,72
4873,14,Senate,2193,ANTI-SALVAGING LAW,2008-04-23,"[Trillanes IV, Antonio ""Sonny"" F, ]","AN ACT QUALIFYING SALVAGING OR EXTRAJUDICIAL KILLING BY ANY PUBLIC OFFICER, PERSON IN AUTHORITY OR AGENT OF A PERSON IN AUTHORITY AS A HEINOUS CRIME, IMPOSING THE DEATH PENALTY THEREFOR AND FOR OTHER PURPOSES",National,Heinous Crimes,Justice and Human Rights,anti-salvaging law act qualifying salvaging extrajudicial killing public officer person authority agent person authority heinous crime imposing death penalty therefor purpose,1,6,84
2966,14,Senate,286,SUSPENDING PAYMENT OF ESTATE TAX ON FAMILY HOME BY SURVIVING SPOUSE IN CASE OF DEATH OF SPOUSE,2007-06-30,"[Gordon, Richard ""Dick"" J, ]","AN ACT AMENDING SECTION 86(A) (4) OF REPUBLIC ACT NO. 8424, AS AMENDED, OTHERWISE KNOWN AS THE NATIONAL INTERNAL REVENUE CODE OF 1997",National,"National Internal Revenue Code (NIRC)Family HomeR.A.No. 8424 (Tax Reform Act of 1997), Amendments",Ways and Means,suspending payment estate tax family home surviving spouse case death spouse act amending section 86a 4 republic act 8424 amended otherwise known national internal revenue code 1997,6,14,59


### 5th query: `Climate Change`

#### relevant criteria: `scope`

In [36]:
## sample querying

text_query = 'Climate Change'   ## type a sample query
k = 5                         ## specify no. of results
congress = 14                 ## specify which congress
criteria = 'scope'
IR_search(text_query, k, congress, criteria)

Search term: climate change
Top 5 search results:


Climate Change precision_(scope): 1.000
Climate Change recall_(scope): 0.00102


,congress,type,bill_num,title,filed_on,filed_by,long_title,scope,subjects,primary_committee,combined_title,clustering_ward_a,clustering_ward_b,clustering_ward_c
5121,14,Senate,2441,CLIMATE CHANGE EDUCATION ACT OF 2008,2008-07-14,"[Lapid, Manuel ""Lito"" M, ]",AN ACT INSTITUTIONALIZING THE CLIMATE CHANGE EDUCATION AND AWARENESS PROGRAM AND FOR OTHER PURPOSES,National,Climate Change,"Basic Education, Arts and Culture",climate change education act 2008 act institutionalizing climate change education awareness program purpose,0,2,4
5068,14,Senate,2388,CLIMATE CHANGE EDUCATIONAL PROGRAM ACT,2008-06-10,"[Defensor Santiago, Miriam]",AN ACT ESTABLISHING THE CLIMATE CHANGE EDUCATIONAL PROGRAM,National,Climate Change,"Environment, Natural Resources and Climate Change",climate change educational program act act establishing climate change educational program,0,2,4
5039,14,Senate,2359,DRINKING WATER ADAPTATION RESEARCH ACT,2008-06-02,"[Defensor Santiago, Miriam]",CLIMATE CHANGE DRINKING WATER ADAPTATION RESEARCH ACT,National,Climate ChangeDrinking Water,"Environment, Natural Resources and Climate Change",climate change drinking water adaptation research act,0,2,4
5016,14,Senate,2336,NATIONAL CLIMATE PROGRAM ACT OF 2008,2008-05-28,"[Defensor Santiago, Miriam]",AN ACT ESTABLISHING THE NATIONAL CLIMATE PROGRAM,National,Climate Change,"Environment, Natural Resources and Climate Change",national climate program act 2008 act establishing national climate program,0,1,24
3935,14,Senate,1255,EXACT CHANGE ACT,2007-07-18,"[Defensor Santiago, Miriam]",AN ACT REQUIRING BUSINESS ESTABLISHMENTS TO GIVE EXACT CHANGE TO CONSUMERS,National,Business/Entrepreneurship,"Trade, Commerce and Entrepreneurship",exact change act act requiring business establishment give exact change consumer,0,2,4


#### relevant criteria: `primary_committee`

In [37]:
## sample querying

text_query = 'Climate Change'   ## type a sample query
k = 5                         ## specify no. of results
congress = 14                 ## specify which congress
criteria = 'primary_committee'
IR_search(text_query, k, congress, criteria)

Search term: climate change
Top 5 search results:


Climate Change precision_(primary_committee): 0.199
Climate Change recall_(primary_committee): 0.00067


,congress,type,bill_num,title,filed_on,filed_by,long_title,scope,subjects,primary_committee,combined_title,clustering_ward_a,clustering_ward_b,clustering_ward_c
5121,14,Senate,2441,CLIMATE CHANGE EDUCATION ACT OF 2008,2008-07-14,"[Lapid, Manuel ""Lito"" M, ]",AN ACT INSTITUTIONALIZING THE CLIMATE CHANGE EDUCATION AND AWARENESS PROGRAM AND FOR OTHER PURPOSES,National,Climate Change,"Basic Education, Arts and Culture",climate change education act 2008 act institutionalizing climate change education awareness program purpose,0,2,4
5068,14,Senate,2388,CLIMATE CHANGE EDUCATIONAL PROGRAM ACT,2008-06-10,"[Defensor Santiago, Miriam]",AN ACT ESTABLISHING THE CLIMATE CHANGE EDUCATIONAL PROGRAM,National,Climate Change,"Environment, Natural Resources and Climate Change",climate change educational program act act establishing climate change educational program,0,2,4
5039,14,Senate,2359,DRINKING WATER ADAPTATION RESEARCH ACT,2008-06-02,"[Defensor Santiago, Miriam]",CLIMATE CHANGE DRINKING WATER ADAPTATION RESEARCH ACT,National,Climate ChangeDrinking Water,"Environment, Natural Resources and Climate Change",climate change drinking water adaptation research act,0,2,4
5016,14,Senate,2336,NATIONAL CLIMATE PROGRAM ACT OF 2008,2008-05-28,"[Defensor Santiago, Miriam]",AN ACT ESTABLISHING THE NATIONAL CLIMATE PROGRAM,National,Climate Change,"Environment, Natural Resources and Climate Change",national climate program act 2008 act establishing national climate program,0,1,24
3935,14,Senate,1255,EXACT CHANGE ACT,2007-07-18,"[Defensor Santiago, Miriam]",AN ACT REQUIRING BUSINESS ESTABLISHMENTS TO GIVE EXACT CHANGE TO CONSUMERS,National,Business/Entrepreneurship,"Trade, Commerce and Entrepreneurship",exact change act act requiring business establishment give exact change consumer,0,2,4


### Summary

**Summary of Precision Values from Manual and Automated**<br>

|query|No. of Results|No. of Relevant Results(Manual)|Precision (Manual)|Precision_Scope (IR)|Precision_PrimaryCommittee (IR)|
|-----|--------------|-------------------------------|------------------|--------------------|-------------------------------|
|`minimum wage`| 5 | 5 | 1.00 | 1.00 | 0.214 |
`overseas filipino workers`| 5 | 5 | 1.00 | 1.00 | 0.200 |
`illegal drugs`| 5 | 4 | 0.80 | 1.00 |  0.240 |
`death penalty`| 5 | 2 | 0.40 | 1.00 | 0.201 |
`climate change`| 5 | 4 | 0.80 | 1.00 | 0.199 |